In [25]:
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
import matplotlib  as mpl
import h5py
import matplotlib.colors as colors
from tqdm import tqdm
from scipy import stats
import cmasher as cmr
import seaborn as sns
import warnings
import multiprocessing as mp
from functools import partial
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

from matplotlib.colors import Normalize

import matplotlib.cm as cm
from matplotlib.cm import ScalarMappable

from matplotlib.colors import LogNorm

In [26]:
from msafit.model.config import get_default_config
params = get_default_config()
params['instrument']['filter'] = "290"
params['instrument']['disperser'] = "G235H"

from msafit.fpa import PSFLib
psfs = PSFLib('1x3_PRISM_Q3_PSFLib.fits')
params["grid"]["x_grid"] = psfs.psf_x
params["grid"]["y_grid"] = psfs.psf_y
params["grid"]["wave_grid"] = psfs.psf_wave

from msafit.model import Sersic
# create a model cube I(x,y,lambda)
model = Sersic(params)
model(params["grid"]["wave_grid"])

_param_dict = model._param_dict
x_grid_bins = _param_dict['grid']['x_grid_sky']
y_grid_bins = _param_dict['grid']['y_grid_sky']

In [27]:
def Gal_inf (ID,Snap):

    file_path = f'/virgotng/universe/IllustrisTNG/TNG50-1/output/snapdir_{Snap:03d}/snap_{Snap:03d}.0.hdf5'
    
    with h5py.File(file_path, 'r') as f:
        header = dict( f['Header'].attrs.items())
        
    z = header['Redshift']

    scaling_factor = 1.0 / (1+z)

    hubble_param = header['HubbleParam']
    H0 = hubble_param*100
    Omega_lamb = header['OmegaLambda']
    Omega_baryon = header['OmegaBaryon']
    Omega_0 = header['Omega0']

    basePath = "/virgotng/universe/IllustrisTNG/TNG50-1/output"

    Subhalo = il.groupcat.loadSingle(basePath,Snap, subhaloID = ID)

    Subhalo_gas_part = il.snapshot.loadSubhalo(basePath, snapNum=Snap, id=ID, partType='gas', fields=["StarFormationRate","Masses","Coordinates","Velocities"])

    Starforming_cond = Subhalo_gas_part["StarFormationRate"] > 0

    Coord_gas = (Subhalo_gas_part["Coordinates"][Starforming_cond]-Subhalo["SubhaloPos"]) * scaling_factor / hubble_param

    radius = np.sqrt(np.sum(Coord_gas**2, axis=1))
    
    Rad_cond  = radius < (0.4*Subhalo["SubhaloHalfmassRadType"][0])
    
    velocity_gas = ((Subhalo_gas_part["Velocities"][Starforming_cond])[:,2]*np.sqrt(scaling_factor)) - Subhalo["SubhaloVel"][2]

    SFR = Subhalo_gas_part["StarFormationRate"][Starforming_cond]
    
    return(Coord_gas[Rad_cond],velocity_gas[Rad_cond],SFR[Rad_cond],Omega_0, Omega_lamb, Omega_baryon, H0, z)


def Distances(Omega_0, Omega_baryon, H0, z):

    cosmo = FlatLambdaCDM(H0=H0, Om0=Omega_0, Ob0=Omega_baryon)

    D_A = cosmo.angular_diameter_distance(z).to(u.Mpc)

    D_A_kpc = D_A.to(u.kpc)

    D_L = cosmo.luminosity_distance(z)

    D_A_val = D_A_kpc.value

    D_L = D_L.value
    
    return(D_A_val,D_L)


def Flux(SFR,Dl):

    L_Halpha = SFR/7.9e-42 #erg . s-1

    D_l = Dl  * 3.08567758e24 #Mpc to cm

    FHalpha = L_Halpha / (4*np.pi*(D_l**2)) # erg.cm-2
    
    return (FHalpha)


def Bins(Coord,v_z,FHalpha,DA,speed_bin_size,z):

    # Convert angular resolution from arcseconds to radians
    angular_resolution = 0.02 # Arcsec

    angular_res_rad = angular_resolution * (np.pi/(180*3600))

    pixel_size = angular_res_rad * DA # d = theta * DA in kpc
    x_coord_min = np.min(Coord[:,0])
    x_coord_max = np.max(Coord[:,0])
    y_coord_min = np.min(Coord[:,1])
    y_coord_max = np.max(Coord[:,1])
    
    num_bins_x = int(np.ceil((x_coord_max - x_coord_min) / pixel_size))
    num_bins_y = int(np.ceil((y_coord_max - y_coord_min) / pixel_size))

    bin_width_x = (x_coord_max - x_coord_min) / num_bins_x
    bin_width_y = (y_coord_max - y_coord_min) / num_bins_y

    bin_edges_x = np.linspace(x_coord_min, x_coord_max, num_bins_x + 1)
    bin_edges_y = np.linspace(y_coord_min, y_coord_max, num_bins_y + 1)

    bin_width_vz = speed_bin_size
    vz_min = np.nanmin(v_z)
    vz_max = np.nanmax(v_z)

    num_bins_v_z = int(np.ceil((vz_max - vz_min) / bin_width_vz))
    bin_edges_v_z = np.linspace(vz_min, vz_max, num_bins_v_z + 1)

    Bin_edges = [bin_edges_x,bin_edges_y,bin_edges_v_z]
    
    return (Bin_edges)   


def Delta_Lambda(v_z):
    
    H_alpha_wave = 656.46 #nm

    c = 299792 #km/s

    delt_lamb = (v_z/c)*H_alpha_wave

    return(delt_lamb)
    

def Cube_maker (Coord,Lambda,flux,Bin_edges,bin_edges_lambda) :
    # Digitize the coordinates
    x_indices = np.digitize(Coord[:,0], Bin_edges[0]) - 1  # Subtract 1 to get zero-based index
    y_indices = np.digitize(Coord[:,1], Bin_edges[1]) - 1  # Subtract 1 to get zero-based index
    delta_lambda_indices = np.digitize(Lambda, bin_edges_lambda) - 1  # Subtract 1 to get zero-based index

    nb_x_bins = len(Bin_edges[0]) - 1
    nb_y_bins = len(Bin_edges[1]) - 1
    nb_delta_lambda_bins = len(bin_edges_lambda) - 1
    
    # Create the 3D array for accumulating flux values
    cube_3d = np.zeros((nb_x_bins, nb_y_bins, nb_delta_lambda_bins))
    
    # Accumulate flux into the bins
    for i in range(len(Coord[:,0])):
        if 0 <= x_indices[i] < nb_x_bins and 0 <= y_indices[i] < nb_y_bins and 0 <= delta_lambda_indices[i] < nb_delta_lambda_bins:
            cube_3d[x_indices[i], y_indices[i], delta_lambda_indices[i]] += flux[i]
    
    return (cube_3d)

In [ ]:
ID = 1
Snap = 50
#Snap_nums = [99, 50, 33, 25, 21, 17, 13] #z = 0,1,2,3,4,5,6

speed_bin_size = float(input("Enter the bin size of speed in (km/s): "))

Coord_gas,velocity_gas,SFR,Omega_0, Omega_lamb, Omega_baryon, H0, Z = Gal_inf(ID,Snap)
Dl = Distances(Omega_0, Omega_baryon, H0, Z)
FHalpha = Flux(SFR,Dl)
Bin_edges = Bins(Coord_gas,velocity_gas,FHalpha,DA,speed_bin_size,Z)
Lambda = Delta_Lambda(velocity_gas)
bin_edges_lambda = Delta_Lambda(Bin_edges[2])
cube = Cube_maker(Coord_gas,Lambda,FHalpha,Bin_edges,bin_edges_lambda)